
Following Tools are functions that agents can call to interact with external systems. For SQL databases, tools include:

- **sql_db_query**: Execute SQL queries
- **sql_db_schema**: Get schema details for specific tables
- **sql_db_list_tables**: list of tables in the database
- **sql_db_query_checker**: Validate SQL syntax
- **list_all_tables**: List tables across all schemas (not just dbo)
- **get_table_schema**: Custom tool to get schema details for specific tables
- **get_database_summary**: Custom tool 
- **find_relevant_tables**: Custom tool for vector search for relevant tables
- **search_tables_by_keyword**: Custom tool for Search tables by keyword

In [1]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

## 2. List All Available Database Tools

This section loads and displays all tools available to the SQL agent.

In [2]:
from src.agents.tools import db_tool_manager
from src.config.models import get_llm

# Initialize the language model
llm = get_llm()

# Get all database tools from the tool manager
# The tool manager configures tools for multi-schema database access
tools = db_tool_manager.get_tools(llm)

print(f"✅ {len(tools)} tools ready\n")
print("Available Database Tools:")
print("=" * 80)

# Display each tool with its name and description
for tool in tools:
    print(f"\n📌 Tool: {tool.name}")
    # Truncate description for readability
    print(f"   Description: {tool.description}...")
    print("-" * 80)

c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ 9 tools ready

Available Database Tools:

📌 Tool: sql_db_query
   Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields....
--------------------------------------------------------------------------------

📌 Tool: sql_db_schema
   Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3...
--------------------------------------------------------------------------------

📌 Tool: sql_db_list_tables
   Description: Input is an empty string, output is a comma-separated list of tables in the database..

## 3. Test Available Database Tools


In [3]:
# Find specific tools by name
list_tables_tool = next((t for t in tools if t.name == "list_all_tables"), None)
schema_tool = next((t for t in tools if t.name == "sql_db_schema"), None)
custom_schema_tool = next((t for t in tools if t.name == "get_table_schema"), None)
query_tool = next((t for t in tools if t.name == "sql_db_query"), None)
find_relevant_tool = next((t for t in tools if t.name == "find_relevant_tables"), None)

print("=== Testing list_all_tables Tool ===")
if list_tables_tool:
    # List all tables across all schemas
    tables = list_tables_tool.invoke("")
    print(f"Found {len(tables) if isinstance(tables, list) else 'N/A'} tables")
    print("Sample tables:", tables[:5] if isinstance(tables, list) else str(tables)[:200])
else:
    print("❌ list_all_tables not found")

print("\n=== Testing sql_db_schema Tool===")
if schema_tool:
    # Get schema for a specific table (use an actual table from your database)
    try:
        schema_info = schema_tool.invoke("Sales.Customer")
        print("Schema information:")
        print(schema_info[:500] + "..." if len(schema_info) > 500 else schema_info)
    except Exception as e:
        print(f"Note: Replace 'Sales.Customer' with an actual table name from your database")
        print(f"Error: {e}")
else:
    print("❌ sql_db_schema not found")

print("\n=== Testing get_table_schema Tool===")
if custom_schema_tool:
    # Get schema for a specific table (use an actual table from your database)
    try:
        schema_info = custom_schema_tool.invoke("Sales.Customer")
        print("Schema information:")
        print(schema_info[:500] + "..." if len(schema_info) > 500 else schema_info)
    except Exception as e:
        print(f"Note: Replace 'Sales.Customer' with an actual table name from your database")
        print(f"Error: {e}")
else:
    print("❌ get_table_schema not found")

print("\n=== Testing find_relevant_tables Tool===")
if find_relevant_tool:
    # Use vector search to find relevant tables
    result = find_relevant_tool.invoke("customer and sales information")
    print("Relevant tables found:")
    print(result[:500] + "..." if len(result) > 500 else result)
else:
    print("❌ find_relevant_tables not found")


INFO: Found 14 tables
ERROR: Error getting schema for 'Sales.Customer': Sales.Customer


=== Testing list_all_tables Tool ===
Found 14 tables
Sample tables: ['public.categories', 'public.customer_customer_demo', 'public.customer_demographics', 'public.customers', 'public.employee_territories']

=== Testing sql_db_schema Tool===
Schema information:
Error: table_names {'Sales.Customer'} not found in database

=== Testing get_table_schema Tool===
Schema information:
Error getting schema for 'Sales.Customer': Sales.Customer

=== Testing find_relevant_tables Tool===
Searching schema vector store for question: customer and sales information


c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\vector_retriver_tools.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Relevant tables found:
--- Relevant Table 1: Sales.Customer ---
Table: Sales.Customer

Schema Details:
Table: Sales.Customer

Columns:
  - CustomerID: INTEGER NOT NULL
  - PersonID: INTEGER NULL
  - StoreID: INTEGER NULL
  - TerritoryID: INTEGER NULL
  - AccountNumber: VARCHAR(10) COLLATE "SQL_Latin1_General_CP1_CI_AS" NOT NULL
  - rowguid: UNIQUEIDENTIFIER NOT NULL
  - ModifiedDate: DATETIME NOT NULL

Primary Key: CustomerID

Foreign Keys:
  - PersonID -> Person(BusinessEntityID)
  - TerritoryID -> SalesTerritory(Ter...
